# 合成音声の生成
- ここでは，目的話者の短い音声を参照として，その人の話者性を持つ任意の文章をの発話を生成する方法を概説します．
- この技術はzero-shot voice cloningと呼ばれます．
- ここでは，[CosyVoice](https://github.com/FunAudioLLM/CosyVoice/tree/main)というモデルを用いて合成音声を生成する方法を示します．

## 環境構築
このノートブックを実行する前に、CosyVoice用の環境をセットアップしてください：

In [1]:
!uv sync --extra cosyvoice

Resolved 222 packages in 38ms
Audited 186 packages in 0.84ms


## コードのダウンロード
CosyVoiceのコードをクローンします．

In [2]:
# CosyVoiceをクローン
!git clone --recursive https://github.com/FunAudioLLM/CosyVoice.git

Cloning into 'CosyVoice'...
remote: Enumerating objects: 3025, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 3025 (delta 117), reused 75 (delta 75), pack-reused 2843 (from 3)
Receiving objects: 100% (3025/3025), 2.07 MiB | 3.24 MiB/s, done.
Resolving deltas: 100% (1755/1755), done.
Submodule 'third_party/Matcha-TTS' (https://github.com/shivammehta25/Matcha-TTS.git) registered for path 'third_party/Matcha-TTS'
Cloning into '/home/tenk9/ShiotaLab/B3_seminar_2025_sample_codes/handson/CosyVoice/third_party/Matcha-TTS'...
remote: Enumerating objects: 1081, done.        
remote: Counting objects: 100% (556/556), done.        
remote: Compressing objects: 100% (201/201), done.        
remote: Total 1081 (delta 449), reused 355 (delta 355), pack-reused 525 (from 1)        
Receiving objects: 100% (1081/1081), 64.10 MiB | 7.28 MiB/s, done.
Resolving deltas: 100% (538/538), done.
Submodule path 'third_party/Matcha-TTS': che

## モデル本体をダウンロード

In [12]:
!mkdir -p pretrained_models

# # Hugging Faceからモデルをダウンロード
# !git clone https://huggingface.co/FunAudioLLM/CosyVoice2-0.5B pretrained_models/CosyVoice2-0.5B

# modelscopeからDL
from modelscope import snapshot_download
snapshot_download('iic/CosyVoice2-0.5B', local_dir='pretrained_models/CosyVoice2-0.5B')

2025-12-10 10:32:28,841 DEBUG Starting new HTTPS connection (1): www.modelscope.cn:443


2025-12-10 10:32:29,095 DEBUG Incremented Retry for (url='/api/v1/models/iic/CosyVoice2-0.5B/revisions'): Retry(total=1, connect=2, read=1, redirect=None, status=None)
2025-12-10 10:32:29,097 WARNING Retrying (Retry(total=1, connect=2, read=1, redirect=None, status=None)) after connection broken by 'ConnectionResetError(104, 'Connection reset by peer')': /api/v1/models/iic/CosyVoice2-0.5B/revisions
2025-12-10 10:32:29,099 DEBUG Starting new HTTPS connection (2): www.modelscope.cn:443
2025-12-10 10:32:43,153 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/iic/CosyVoice2-0.5B/revisions HTTP/1.1" 200 None
2025-12-10 10:32:44,071 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/iic/CosyVoice2-0.5B/repo/files?Revision=master&Recursive=True HTTP/1.1" 200 None
2025-12-10 10:32:44,089 DEBUG Starting new HTTPS connection (1): www.modelscope.cn:443
2025-12-10 10:32:50,053 DEBUG https://www.modelscope.cn:443 "GET /api/v1/models/iic/CosyVoice2-0.5B/repo?Revision=master&FilePath=flow

MaxRetryError: None: Max retries exceeded with url: https://www.modelscope.cn/api/v1/models/iic/CosyVoice2-0.5B/repo?Revision=master&FilePath=flow.pt (Caused by SSLError(MaxRetryError('HTTPSConnectionPool(host=\'www.modelscope.cn\', port=443): Max retries exceeded with url: /api/v1/models/iic/CosyVoice2-0.5B/repo?Revision=master&FilePath=flow.pt (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for \'www.modelscope.cn\'. (_ssl.c:1016)")))')))

## 実行

Qwen2ForCausalLMが見つからないと言われたり，何もしていないのにエラーが出たときは，ipynbのカーネルを再起動してみてください．
一応重いけどCPUでも動くはず．

In [9]:
import sys

sys.path.append("CosyVoice")
sys.path.append("CosyVoice/third_party/Matcha-TTS")
from cosyvoice.cli.cosyvoice import CosyVoice, CosyVoice2
from cosyvoice.utils.file_utils import load_wav
import torchaudio
import os

# modelのインスタンス化
cosyvoice = CosyVoice2(
    "pretrained_models/CosyVoice2-0.5B",
    load_jit=False,
    load_trt=False,
    load_vllm=False,
    fp16=False,
)

# 参照音声を読み込み
prompt_speech_16k = load_wav("./audiofile/ymgt.wav", 16000)

# 合成音声を生成
## 読み上げ文章，発話スタイル等のプロンプトをそれぞれ指定
## うまく出力されないときはこれらを調整する．
## 本来はendofpromptとか入れなくても動くらしいが，入れたほうが安定するので入れています．
## 参考：公式Usage：https://github.com/FunAudioLLM/CosyVoice/blob/main/README.md
## 参考：特殊トークン: https://github.com/FunAudioLLM/CosyVoice/blob/main/cosyvoice/tokenizer/tokenizer.py
reading_texts = [
    "<|jp|> [breath] みなみおおさわキャンパスの法定停電に伴い、 [clucking] 以下の期間はキャンパススクエア・自動証明書発行機を利用できません。 <|endoftext|>",
    "<|en|> An extremely fast Python package and project manager, written in Rust. <|endoftext|>",
]
prompt = "<|jp|> 流暢で早口な，元気な読み上げ音声 <|endofprompt|>"

out_dir = "./CosyVoice_output"
os.makedirs(out_dir, exist_ok=True)

for i, txt in enumerate(reading_texts):
    # outputは一つだけだが，返り値のType的にforで拾ってやる必要がある（他にもやり方はあるが公式Usageに則る）
    for output in cosyvoice.inference_zero_shot(
        txt,
        prompt,
        prompt_speech_16k,
        stream=False,
        text_frontend=False,
    ):
        cloned_speech = output["tts_speech"]

        # 音声を保存
        torchaudio.save(
            f"{out_dir}/out{i}.wav",
            src=cloned_speech,
            sample_rate=cosyvoice.sample_rate,
        )

SafetensorError: Error while deserializing header: header too large

## 音声の確認

In [ ]:
import IPython.display as ipd

# 参照音声を再生
print("参照音声:")
display(ipd.Audio("./audiofile/ymgt.wav"))

# 生成された音声を再生
print("\n生成された音声1 (日本語):")
display(ipd.Audio(f"{out_dir}/out0.wav"))

print("\n生成された音声2 (英語):")
display(ipd.Audio(f"{out_dir}/out1.wav"))

参照音声:



生成された音声1 (日本語):



生成された音声2 (英語):
